In [46]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials


In [ ]:
color_to_day = {
    (0.62352943, 0.77254903, 0.9098039): "Tuesday",
    (0.7058824, 0.654902, 0.8392157): "Wednesday",
    (0.5764706, 0.76862746, 0.49019608): "Thursday",
    (0.91764706, 0.6, 0.6): "Friday",
}

In [47]:
# Yetkilendirme
SHEET_ID = "19M06LpwDMRQzBgTAewlN8aq300ukLw1xVC-ObX1vcgw"  # senin dosyanın ID'si
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
creds = Credentials.from_service_account_file('earningsplaynewsheet-748c7181e867.json', scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Spreadsheet aç
spreadsheet = service.spreadsheets().get(spreadsheetId=SHEET_ID).execute()
sheet_titles = [sheet['properties']['title'] for sheet in spreadsheet['sheets']]

# Sadece tarih olan sheetleri ayır
import re
weekly_sheet_titles = [title for title in sheet_titles if re.match(r"\d{2}/\d{2}/\d{4}", title)]

In [48]:
def read_weekly_sheet(sheet_name):
    # Sheet verisini çek
    sheet = service.spreadsheets().get(
        spreadsheetId=SHEET_ID,
        ranges=sheet_name,
        fields="sheets.data.rowData.values.effectiveFormat.backgroundColor,sheets.data.rowData.values.formattedValue",
        includeGridData=True
    ).execute()
    
    grid_data = sheet['sheets'][0]['data'][0]['rowData']
    
    # Data ve renkleri topla
    data = []
    for idx, row in enumerate(grid_data):
        if 'values' in row and len(row['values']) > 0:
            cells = row['values']
            if len(cells) < 1 or not cells[0].get('formattedValue', '').strip():
                continue

            ticker = cells[0].get('formattedValue', '').strip()
            company_name = cells[1].get('formattedValue', '').strip() if len(cells) > 1 else ''
            price = cells[2].get('formattedValue', '').strip() if len(cells) > 2 else ''
            total_move = cells[3].get('formattedValue', '').strip() if len(cells) > 3 else ''
            move_pct = cells[4].get('formattedValue', '').strip() if len(cells) > 4 else ''
            
            # Arka plan rengi
            color = cells[0].get('effectiveFormat', {}).get('backgroundColor', {})
            r = round(color.get('red', 1.0), 7)
            g = round(color.get('green', 1.0), 7)
            b = round(color.get('blue', 1.0), 7)
            rgb = (r, g, b)

            data.append({
                'Row': idx+1,
                'Ticker': ticker,
                'Company Name': company_name,
                'Price': price,
                'Total Move': total_move,
                'Move %': move_pct,
                'Color': rgb
            })
    
    df = pd.DataFrame(data)
    return df

In [49]:
color_to_day = {
    (0.6235294, 0.772549, 0.9098039): "Tuesday",
    (0.7058824, 0.654902, 0.8392157): "Wednesday",
    (0.5764706, 0.76862746, 0.49019608): "Thursday",
    (0.91764706, 0.6, 0.6): "Friday",
}

In [50]:
def find_day_from_color(rgb):
    for key, day in color_to_day.items():
        if all(abs(c1 - c2) < 0.02 for c1, c2 in zip(rgb, key)):
            return day
    return None

In [63]:
def build_master_rows(weekly_df, week_start):
    master_rows = []

    for _, row in weekly_df.iterrows():
        ticker = row['Ticker']
        company_name = row['Company Name']
        price = row['Price']
        total_move = row['Total Move']
        move_pct = row['Move %']
        color = row['Color']
        
        weekday = find_day_from_color(color)
        
        if weekday is None:
            continue  # Gün bulunamıyorsa geç
        
        day_to_delta = {
            "Tuesday": 1,
            "Wednesday": 2,
            "Thursday": 3,
            "Friday": 4,
        }
        delta_days = day_to_delta.get(weekday, None)
        if delta_days is None:
            continue

        report_date = pd.to_datetime(week_start, dayfirst=True) + pd.Timedelta(days=delta_days)

                # Sadece tarih kısmını alıyoruz
        report_date_only = report_date.date()  # `date()` metodu ile saat kısmını çıkarıyoruz

        master_rows.append({
            'Week': week_start,
            'Ticker': ticker,
            'Company Name': company_name,
            'Price': price,
            'Total Move': total_move,
            'Move %': move_pct,
            'Weekday Reported': weekday,
            'Report_Date': report_date_only  # ⚡⚡ DİKKAT: Burada artık strftime yok
        })

    return master_rows

In [66]:
all_master_rows = []

for weekly_title in weekly_sheet_titles:
    print(f"Processing week: {weekly_title}")
    weekly_df = read_weekly_sheet(weekly_title)
    week_rows = build_master_rows(weekly_df, weekly_title)
    all_master_rows.extend(week_rows)

# Şimdi tüm satırları DataFrame yapalım
master_df = pd.DataFrame(all_master_rows)

Processing week: 21/04/2025
Processing week: 14/04/2025
Processing week: 07/04/2025
Processing week: 31/03/2025
Processing week: 24/03/2025
Processing week: 17/03/2025
Processing week: 10/03/2025
Processing week: 03/03/2025
Processing week: 24/02/2025
Processing week: 17/02/2025
Processing week: 10/02/2025
Processing week: 03/02/2025
Processing week: 27/01/2025
Processing week: 20/01/2025
Processing week: 13/01/2025
Processing week: 06/01/2025


In [ ]:
# Yeni haftayı oku
new_week_title = "29/04/2025"  # Örneğin
new_week_df = read_weekly_sheet(new_week_title)
new_week_rows = build_master_rows(new_week_df, new_week_title)

# Yeni haftayı eski master'a ekle
new_rows_df = pd.DataFrame(new_week_rows)
master_df = pd.concat([master_df, new_rows_df], ignore_index=True)

In [67]:
import gspread
from gspread_dataframe import set_with_dataframe

# Yetkilendirme
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("earningsplaynewsheet-748c7181e867.json", scope)
client = gspread.authorize(creds)

# Spreadsheet aç
spreadsheet = client.open_by_key("19M06LpwDMRQzBgTAewlN8aq300ukLw1xVC-ObX1vcgw")

# Master2025 tabını seç (veya oluştur)
try:
    master_sheet = spreadsheet.worksheet("Master2025")
    master_sheet.clear()
except:
    master_sheet = spreadsheet.add_worksheet(title="Master2025", rows="1000", cols="20")

# DataFrame'i Master2025 tabına yaz
set_with_dataframe(master_sheet, master_df)